### 재무정보(전체 재무제표) - 단일 회사(상장기업) - 사업보고서

In [12]:
fiscal_year = '2018'
report_code = '11011'      # 사업보고서
listed_corp_codes = \
['01208885',
 '00787376',
 '00128555',
 '00130684']

In [13]:
import requests
import pandas as pd
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

In [14]:
def dart_report_exist(cursor, report_code, fiscal_year, corp_code):
    query = f'SELECT EXISTS (SELECT * FROM dart_report_detail \
                              WHERE reprt_code = {report_code} and bsns_year = {fiscal_year} and corp_code = {corp_code})'

    cursor.execute(query)
    row = cursor.fetchone()
    data_exist = row[0]     # 저장된 데이터의 유무(1 - 데이터 있음)
    return data_exist

In [15]:
# 재무제표 가져오기
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json'
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'
df_fi_all = pd.DataFrame()

for corp_code in listed_corp_codes:
    data_exist = dart_report_exist(cur, report_code, fiscal_year, corp_code)
    if data_exist == 1:      # 데이터 존재하면
        pass
    else:                   # 데이터 존재하지 않으면
        params = {
            'corp_code': corp_code,
            'bsns_year': fiscal_year,
            'reprt_code': report_code,  
            'fs_div': 'OFS',            # OFS:재무제표
            'crtfc_key': api_key,
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:             # URL GET 200 : 정상
            print("URL GET Error")
        if response.json()['status'] == '000':      # 데이터 유무 '000' : 정상
            # 데이터프레임 형태로 변환
            df_fi = pd.json_normalize(response.json()['list'])
            if not df_fi.empty:
                df_fi_all = pd.concat([df_fi_all, df_fi], ignore_index=True)

conn.close()

In [16]:
df_fi_all.empty

False

In [18]:
df_fi_all

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,currency,thstrm_add_amount,bfefrmtrm_nm,bfefrmtrm_amount
0,20190401004969,11011,2018,01208885,BS,재무상태표,ifrs_CurrentAssets,유동자산,-,제 2 기,441723741978,제 1 기,374489352903,1,KRW,NaN,NaN,NaN
1,20190401004969,11011,2018,01208885,BS,재무상태표,ifrs_CashAndCashEquivalents,현금및현금성자산,-,제 2 기,116124452661,제 1 기,58793028532,2,KRW,NaN,NaN,NaN
2,20190401004969,11011,2018,01208885,BS,재무상태표,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,-,제 2 기,71500000000,제 1 기,46500000000,3,KRW,NaN,NaN,NaN
3,20190401004969,11011,2018,01208885,BS,재무상태표,ifrs_CurrentFinancialAssetsAtAmortisedCost,단기상각후원가채무증권,-,제 2 기,22586662,제 1 기,38570548,4,KRW,NaN,NaN,NaN
4,20190401004969,11011,2018,01208885,BS,재무상태표,dart_ShortTermTradeReceivable,매출채권,-,제 2 기,248781273585,제 1 기,263643633291,5,KRW,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,20190401003436,11011,2018,00130684,SCE,자본변동표,ifrs_Equity,기말자본,자본 [member]|기타자본항목,제 36 기,-45337353977,제 35 기,-45274180787,10,KRW,NaN,제 34 기,-45243936977
645,20190401003436,11011,2018,00130684,SCE,자본변동표,ifrs_Equity,기말자본,자본 [member]|기타포괄손익누계액 [member],제 36 기,-2640391647,제 35 기,-2062206189,10,KRW,NaN,제 34 기,-2091471985
646,20190401003436,11011,2018,00130684,SCE,자본변동표,ifrs_Equity,기말자본,자본 [member]|이익잉여금 [member],제 36 기,914676820530,제 35 기,893490590682,10,KRW,NaN,제 34 기,858503853043
647,20190401003436,11011,2018,00130684,SCE,자본변동표,ifrs_Equity,기말자본,자본 [member]|자본금 [member],제 36 기,35000000000,제 35 기,35000000000,10,KRW,NaN,제 34 기,35000000000


In [29]:
if not df_fi_all.empty:
    # 불필요한 컬럼 삭제
    df_fi_all.drop(labels=['account_id'], axis=1, inplace=True)
    
    # 문자열 컬럼에 있는 null값을 ''로 변환
    df_fi_all['thstrm_nm'].fillna('', inplace=True)
    df_fi_all['frmtrm_nm'].fillna('', inplace=True)
    df_fi_all['bfefrmtrm_nm'].fillna('', inplace=True)

    # 금액 컬럼에 있는 null값을 ''로 변환
    df_fi_all['thstrm_amount'].fillna('', inplace=True)
    df_fi_all['frmtrm_amount'].fillna('', inplace=True)
    df_fi_all['bfefrmtrm_amount'].fillna('', inplace=True)
    df_fi_all['thstrm_add_amount'].fillna('', inplace=True)

    # 금액 컬럼에 있는 ''을 '0'로 변환(저장 시에 숫자 형식으로 변환을 위해서임)
    df_fi_all = df_fi_all.replace({'thstrm_amount' : '',
                                  'frmtrm_amount' : '',
                                  'bfefrmtrm_amount' : '',
                                  'thstrm_add_amount' : ''}, '0')
    
    # 타입 변환
    df_fi_all = df_fi_all.astype({'thstrm_amount' : 'float',
                                  'frmtrm_amount' : 'float',
                                  'bfefrmtrm_amount' : 'float',
                                  'thstrm_add_amount' : 'float'})
    
    df_fi_all = df_fi_all.astype({'ord' : 'int'})
    
    # df_fi_all.to_excel('D:\PythonProject\data-gatherer\dart\data\co_fi_all_multiple.xlsx')

### 테이블에 저장

In [30]:
if not df_fi_all.empty:
    conn = dbconnect.db_connect("DEMO_DW")
    cur = conn.cursor()
    # 테이블 컬럼 순서대로 항목 재정렬
    df_fi_save = df_fi_all[['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'sj_div', 'sj_nm', 'account_nm', 'account_detail', \
                        'thstrm_nm', 'thstrm_amount', 'thstrm_add_amount', 'frmtrm_nm', 'frmtrm_amount', 'bfefrmtrm_nm', 'bfefrmtrm_amount', \
                        'ord', 'currency']]
    # df_fi_save.to_excel('D:\PythonProject\data-gatherer\dart\data\df_fi_save.xlsx')

    for row in df_fi_save.itertuples():
        sql = "insert into dart_report_detail(rcept_no, reprt_code, bsns_year, corp_code, sj_div, sj_nm, account_nm, account_detail, \
                                            thstrm_nm, thstrm_amount, thstrm_add_amount, frmtrm_nm, frmtrm_amount, bfefrmtrm_nm, bfefrmtrm_amount, \
                                            ord, currency) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], \
                        row[10], row[11], row[12], row[13], row[14], row[15], row[16], row[17]))

    conn.commit()
    conn.close()